# Step47, 소프트맥스 함수와 교차 엔트로피 오차 

지금까지 신경망을 사용하여 회귀문제를 풀었다.

다중 클래스 분류(multi-class classification) : '여러 클래스'로 '분류'하는 문제  
분류 대상이 여러 가지 클래스 중 어디에 속하는지 추정

다중 클래스 분류를 구현해 본다.

## 47 부록, get_item 함수 구현

Variable의 다차원 배열 중에서 일부를 슬라이스 하여야하는데 이때 사용한는 것이 get_item이다.  

먼저 get_item 함수를 구현해본다

In [1]:
# dezero/functions.py
from dezero import Function

class GetItem(Function):
    def __init__(self, slices):
        self.slices = slices        # 슬라이스 연산을 수행하는 인수 slices를 받아 인스턴스 변수에 저장
    
    def forward(self, x):           
        y = x[self.slices]          # 단순히 slices 변수를 이용해 원소를 추출하고 있다
        return y 
    
    def backward(self, gy):         # 슬라이스 조작에 대응하는 역전파 계산이 DeZero 함수중에 없으므로
        x, = self.inputs
        f = GetItemGrad(self.slices, x.shape)   # 새로 클래스를 만들어 사용해야 한다
        return f(gy)

def get_item(x, slices):
    return GetItem(slices)(x)

초기화 시 슬라이스 연산을 수행하는 인수 slices를 받아 인스턴스 변수에 저장하고  
forward(x) 메서드에서는 단순히 이 변수를 이용해 원소를 추출하고 있다.  
그런데 슬라이스 조작에 대응하는 역전파 계산은 DeZero 함수 중에는 없다.  
그래서 별도로 GetItemGrad라는 새로운 DeZero 함수 클래스를 제공한다.

즉, GetItemGrad의 순전파가 GetItem의 역전파에 대응하도록 구현하였다.

GetItemGrad 클래스 코드를 살펴보자.

In [2]:
import numpy as np

# dezero/functions.py
class GetItemGrad(Function):
    def __init__(self, slices, in_shape): # 초기화 메서드에서 슬라이스 연산 인수(slices)와 입력 데이터의 모양(in_shape)
        self.slices = slices
        self.in_shape = in_shape

    def forward(self, gy):      
        gx = np.zeros(self.in_shape)    # 입력용 기울기로서 원소가 모두 0인 다차원 배열 gx를 준비
        np.add.at(gx, self.slices, gy)  # gx의 원소 중 self.slices로 지정한 위치에 gy가 더해진다
        return gx 
    
    def backward(self, ggx):
        return get_item(ggx, self.slices)

초기화 메서드에서 슬라이스 연산 인수(slices)와 함께 입력 데이터의 모양(in_shape)을 받는다.  
그리고 주 계산(forward)에서는 입력용 기울기로서 원소가 모두 0인 다차원 배열 gx를 준비한 다음  
np.add.at(gx,self.slices,gy)를 실행  
그 결과 gx의 원소 중 self.slices로 지정한 위치에 gy가 더해진다.

np.add.at 함수 사용법을 살펴본다.

In [3]:
import numpy as np 
a = np.zeros((2,3))
print(a)

b = np.ones((3,))
print(b)

slices = 1
np.add.at(a,slices,b)
print(a)

[[0. 0. 0.]
 [0. 0. 0.]]
[1. 1. 1.]
[[0. 0. 0.]
 [1. 1. 1.]]


다음으로 np.add.at 함수에 대응하는 역전파를 구현해야한다.  
그런데 get_item함수가 그 일을 해준다.

GetItem의 backward는 GetItemGrad 
GetItemGrad의 backward는 GetItem

## 47.1 슬라이스 조작 함수 

다시 돌아와서 get_item 함수가 준비되었고 get_item 함수 사용 예를 보자

In [4]:
import numpy as np 
from dezero import Variable
import dezero.functions as F 

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.get_item(x,1) # (2,3) shape의 x에서 1번째 행의 원소를 추출
print(y)

variable([4 5 6])


이와 같이 get_item 함수는 Variable의 다차원 배열 중에서 일부를 슬라이스하여 뽑아준다

In [5]:
y.backward()
print(x.grad)

variable([[0. 0. 0.]
          [1. 1. 1.]])


슬라이스로 인한 '계산'은 다차원 배열의 데이터 일부를 수정하지 않고 전달하는 것  
따라서  
그 역전파는 원래의 다차원 배열에서 데이터가 추출된 위치에 해당 기울기를 설정, 그 외에는 0으로 설정(그대로 옮기기만 한거니깐)

또한 get_item 함수를 사용하면 인덱스를 반복 지정하여 동일한 원소를 여러번 때낼 수 있다.

In [6]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
indices = np.array([0,0,1])
y = F.get_item(x, indices)
print(y)

variable([[1 2 3]
          [1 2 3]
          [4 5 6]])


get_item 함수를 Variable의 메서드로도 사용할 수 있게 특수 메서드로 설정 

In [7]:
Variable.__getitem__ = F.get_item   # Variable의 메서드로 설정 

y = x[1]
print(y)

y = x[:,2]
print(y)

variable([4 5 6])
variable([3 6])


이렇게 하면 x\[1\]이나 x\[:,2\]등의 기법을 사용할 때도 get_item 함수가 불린다.  
게다가 이 슬라이스 작업의 역전파도 올바르게 이루어진다.

이 특수 메서드 설정은 dezero/core.py의 setup_variable 함수에 넣는다.

~~~python
def setup_variable():
    Variable.__getitem__ = dezero.functions.get_item
~~~

이것으로 Variable 인스턴스를 자유롭게 슬라이스 할 수 있게 되었다.

## 47.2 소프트맥스 함수 

다중 클래스 분류를 신경망으로 하게 되면  
선형회귀때 이용한 신경망을 그대로 사용할 수 있다.  
--> 앞서 MLP 클래스로 구현해둔 신경망을 그대로 이용할 수 있다는 뜻

In [8]:
from dezero.models import MLP 

model = MLP((10,3))

x = np.array([[0.2, -0.4]])
y = model(x)
print(y)

variable([[ 0.82225048  0.09579651 -0.32184872]])


x의 shape이 (1,2)인 샘플 데이터 하나 --> 원소가 2개인 2차원 벡터   
신경망의 출력 형태 (1,3), 3차원 벡터의 원소 각각이 하나의 클래스에 해당   

출력된 벡터에서 값이 가장 큰 원소의 인덱스가 이 모델이 분류한(정답이라고 예측한) 클래스이다.  
2번 원소의 값이 가장 크기 때문에 2번 클래스로 분류 

하지만 신경망의 출력이 단순히 '수치'이며 이 수치를 '확률'로 변환해줘야 한다.  
이것을 해주는 것이 소프트맥스 함수(softmax function)

$$p_k = \frac{exp(y_k)}{\sum_{i=1}^{n} exp(y_i)} \qquad \qquad \qquad \qquad  \qquad (1)$$

소프트맥스 함수의 입력 $y_k$ 총 n개라고 가정(n : '클래스 수')

(1)은 k번째 출력 $p_k$를 구하는 계산식을 나타냄

입력 데이터가 하나인 경우의 소프트맥스 함수 구현

In [9]:
from dezero import Variable, as_variable
import dezero.functions as F 

def softmax1d(x):
    x = as_variable(x)  # x가 ndarray 인스턴스인 경우 Variable 인스턴스로 변환
    y = F.exp(x)
    sum_y = F.sum(y)
    return y / sum_y 


In [10]:
x = Variable(np.array([[0.2,-0.4]]))
y = model(x)
p = softmax1d(y)

print(y)
print(p)    # '확률'로 변환

variable([[ 0.82225048  0.09579651 -0.32184872]])
variable([[0.55489844 0.26836047 0.17674109]])


배치 데이터에 소프트맥스 함수를 적용할 수 있도록 확장

In [11]:
# dezero/functions.py 

def softmax_simple(x, axis=1):
    x = as_variable(x)
    y = exp(x)
    sum_y = sum(y, axis=axis, keepdims=True)
    return y / sum_y

위 코드들은 간단한 구현  
가장 좋은 방법은 Function 클래스를 상속하여 Softmax 클래스를 구현하고 파이썬 함수로 Softmax를 구현

In [12]:
# functions.py
from dezero import Function

class Softmax(Function):
    def __init__(self, axis=1):
        self.axis = axis

    def forward(self, x):
        y = x - x.max(axis=self.axis, keepdims=True)
        y = np.exp(y)
        y /= y.sum(axis=self.axis, keepdims=True)
        return y

    def backward(self, gy):
        y = self.outputs[0]()
        gx = y * gy
        sumdx = gx.sum(axis=self.axis, keepdims=True)
        gx -= y * sumdx
        return gx


def softmax(x, axis=1):
    return Softmax(axis)(x)

## 47.3 교차 엔트로피 오차 

선형 회귀에서는 손실 함수로 mean_squared_error(평균제곱오차)이용  
다중 클래스 분류에 적합한 손실 함수 : 교차 엔트로피 오차(cross entropy error)

$$L = - \sum_k t_k \log p_k$$

$t_k$ : 정답 데이터의 k차원째 값을 나타냄  
정답 데이터의 각 원소는 정답에 해당하는 클래스면 1, 그렇지 않으면 0으로 기록 : one-hot vector 방식 

$t = (0,0,1)$이고 $p = (p_0,p_1,p_2)$ 인 경우를 대입하면 $L = -\log p_2$ 이다.  
즉 정답 클래스에 해당하는 번호의 확률 p를 추출함으로써 cross entropy error를 계산할 수 있다.  
따라서 정답 데이터에 의해 정답 클래스의 번호가 t로 주어지면 아래와 같이 표현할 수 있다.

$$L = -\log p[t]$$
(p\[t\]는 p에서 t번째 요소만 추출한다는 뜻)

**CAUTION_** 만약 데이터가 N개라면 각 데이터에서 cross entropy error를 구하고, 전체를 더한 다음 N으로 나눈다.  
즉, mean cross entropy error를 구한다.

'softmax funtion'과 'cross entropy error'를 한꺼번에 수행하는 함수를 구현 

In [13]:
# dezero/functions.py 

def sofmax_cross_entropy_simple(x, t):
    x, t = as_variable(x), as_variable(t)
    N = x.shape[0]

    p = softmax(x)  # 또는 softmax_simple(x)
    p = clip(p, 1e-15, 1.0) # log(0)을 방지하기 위해 p의 값을 1e-15 이상으로 한다. 
    log_p = log(p)  # log는 DeZero 함수 
    tlog_p = log_p[np.arange(N), t.data]
    y = -1 * sum(tlog_p) / N
    return y


인수 x: 신경망에서 소프트맥스 함수를 적용하기 전의 출력  
인수 t: 정답 데이터, 정답 클래스의 번호(레이블)가 주어진다고 가정  

p = softmax(x)에서 p의 원소값은 0이상 1이하 (0을 log에 건내면 오류가 발생 clip함수 사용)
log 계산 수행
np.arange(N): \[0,1,...,N-1\] 형태의 ndarray 인스턴스를 생성  
log_p\[np.arange(N), t.data\] 코드는 log_p\[0, t.data\[0\]\], log_p\[1, t.data\[1\]\], ..... 와 정답 데이터 (t.data)에 대응하는 모델을 출력을 구하고 그 값을 1차원 배열에 담아줌

In [14]:
x = np.array([[0.2,-0.4], [0.3,0.5], [1.3,-3.2], [2.1, 0.3]])
t = np.array([2, 0, 1, 0])
y = model(x)
loss = F.softmax_cross_entropy_simple(y,t)
print(loss)

variable(1.085969860341805)


In [15]:
class Clip(Function):
    def __init__(self, x_min, x_max):
        self.x_min = x_min
        self.x_max = x_max

    def forward(self, x):
        y = np.clip(x, self.x_min, self.x_max)
        return y

    def backward(self, gy):
        x, = self.inputs
        mask = (x.data >= self.x_min) * (x.data <= self.x_max)
        gx = gy * mask
        return gx


def clip(x, x_min, x_max):
    return Clip(x_min, x_max)(x)

In [16]:
class Log(Function):
    def forward(self, x):
        xp = cuda.get_array_module(x)
        y = xp.log(x)
        return y

    def backward(self, gy):
        x, = self.inputs
        gx = gy / x
        return gx


def log(x):
    return Log()(x)